In [1]:
import numpy as np
from scipy.sparse import csr_matrix, find
import scipy.sparse as sp

In [2]:
triangles_file = 'triangles.txt'
faces = np.loadtxt(triangles_file, dtype=int) - 1
faces = np.sort(faces, axis=1)

In [3]:
print(faces)

[[ 255  258  262]
 [ 255  262  263]
 [ 261  262  263]
 ...
 [ 969 1008 1018]
 [ 246  247  289]
 [1777 1778 1781]]


In [4]:
new_triangles = np.array([
    [90, 94, 263],
    [341, 348, 1256],
    [499, 498, 501]
]) 

faces_update = np.vstack((faces, new_triangles))
print(faces_update)

[[ 255  258  262]
 [ 255  262  263]
 [ 261  262  263]
 ...
 [  90   94  263]
 [ 341  348 1256]
 [ 499  498  501]]


In [5]:
def edge_array(faces):
    edges_count = {}

    for face in faces:
        v1, v2, v3 = face
        edges = [tuple(sorted((v1, v2))), tuple(sorted((v2, v3))), tuple(sorted((v3, v1)))]

        for edge in edges:
            if edge in edges_count:
                edges_count[edge] += 1
            else:
                edges_count[edge] = 1
    
    edges = np.array([(edge[0], edge[1], count) for edge, count in edges_count.items()])
    print(edges)
    return edges

In [6]:
edges = edge_array(faces_update)

[[ 255  258    2]
 [ 258  262    2]
 [ 255  262    2]
 ...
 [ 459 1720    2]
 [ 458  460    2]
 [ 498  499    2]]


In [7]:
def bound0(n):
    boundary_0 = 1 * np.ones((1, n))
    return boundary_0

In [8]:
def compute_boundary_matrices(faces, edges):
    num_faces = faces.shape[0]
    vertices = np.unique(faces)
    num_vertices = len(vertices)
    num_edges = edges.shape[0]

    vertex_index = {vertex: i for i, vertex in enumerate(vertices)}

    faces = np.sort(faces, axis=1)
    edges = np.sort(edges, axis=1)

    B2_data = []
    B2_rows = []
    B2_cols = []
    for i, face in enumerate(faces):
        for j in range(3):
            edge = tuple(sorted((face[j], face[(j + 1) % 3])))
            edge_idx = np.where((edges == edge).all(axis=1))[0][0]
            B2_data.append(1)
            B2_rows.append(edge_idx)
            B2_cols.append(i)
    
    B2 = csr_matrix((B2_data, (B2_rows, B2_cols)), shape=(num_edges, num_faces))

    B1_data = []
    B1_rows = []
    B1_cols = []
    for i, edge in enumerate(edges):
        for vertex in edge:
            B1_data.append(1)
            B1_rows.append(vertex_index[vertex])
            B1_cols.append(i)
    
    B1 = csr_matrix((B1_data, (B1_rows, B1_cols)), shape=(num_vertices, num_edges))

    return B1, B2


In [9]:
v = np.max(edges) + 1
print(v)

1932


In [10]:
bound0(v)

array([[1., 1., 1., ..., 1., 1., 1.]])

In [11]:
e = np.delete(edges, 2, axis=1)
print(e)

[[ 255  258]
 [ 258  262]
 [ 255  262]
 ...
 [ 459 1720]
 [ 458  460]
 [ 498  499]]


In [12]:
bound1,bound2 = compute_boundary_matrices(faces_update, e)

In [13]:
np.sum(bound2 == 1)

np.int64(11568)

In [14]:
print(bound2.shape)

(5784, 3856)


In [15]:
np.sum(bound2 == 1)

np.int64(11568)

In [16]:
print(bound1)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 11568 stored elements and shape (1928, 5784)>
  Coords	Values
  (0, 2376)	1
  (0, 2377)	1
  (0, 2382)	1
  (0, 2383)	1
  (0, 4437)	1
  (0, 4460)	1
  (1, 2386)	1
  (1, 2387)	1
  (1, 2388)	1
  (1, 2389)	1
  (1, 2391)	1
  (2, 2396)	1
  (2, 2397)	1
  (2, 2401)	1
  (2, 2785)	1
  (2, 2846)	1
  (2, 2854)	1
  (3, 2701)	1
  (3, 2703)	1
  (3, 2787)	1
  (3, 2789)	1
  (3, 2847)	1
  (3, 2849)	1
  (4, 2785)	1
  (4, 2786)	1
  :	:
  (1923, 4933)	1
  (1923, 5069)	1
  (1924, 4927)	1
  (1924, 4928)	1
  (1924, 4930)	1
  (1924, 5067)	1
  (1924, 5070)	1
  (1925, 5073)	1
  (1925, 5074)	1
  (1925, 5076)	1
  (1925, 5077)	1
  (1925, 5100)	1
  (1926, 4906)	1
  (1926, 4907)	1
  (1926, 4910)	1
  (1926, 4918)	1
  (1926, 4934)	1
  (1926, 4936)	1
  (1926, 4938)	1
  (1927, 5061)	1
  (1927, 5062)	1
  (1927, 5063)	1
  (1927, 5064)	1
  (1927, 5094)	1
  (1927, 5096)	1


In [17]:
print(bound2)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 11568 stored elements and shape (5784, 3856)>
  Coords	Values
  (0, 0)	1
  (0, 8)	1
  (1, 0)	1
  (1, 21)	1
  (2, 0)	1
  (2, 1)	1
  (3, 1)	1
  (3, 2)	1
  (4, 1)	1
  (4, 5)	1
  (5, 2)	1
  (5, 3)	1
  (6, 2)	1
  (6, 17)	1
  (7, 3)	1
  (7, 4)	1
  (8, 3)	1
  (8, 20)	1
  (9, 4)	1
  (9, 25)	1
  (10, 4)	1
  (10, 7)	1
  (11, 5)	1
  (11, 10)	1
  (12, 5)	1
  :	:
  (5771, 3835)	1
  (5772, 3835)	1
  (5772, 3842)	1
  (5773, 3836)	1
  (5773, 3837)	1
  (5774, 3837)	1
  (5774, 3843)	1
  (5775, 3838)	1
  (5775, 3845)	1
  (5776, 3839)	1
  (5776, 3844)	1
  (5777, 3839)	1
  (5777, 3840)	1
  (5778, 3840)	1
  (5778, 3841)	1
  (5779, 3841)	1
  (5779, 3843)	1
  (5780, 3842)	1
  (5780, 3844)	1
  (5781, 3842)	1
  (5781, 3843)	1
  (5782, 3844)	1
  (5782, 3845)	1
  (5783, 3846)	1
  (5783, 3855)	1


In [20]:
def snf(B_sparse):

    B = B_sparse.toarray().astype(int) % 2
    m, n = B.shape
    i = 0

    while i < min(m, n):
        submatrix = B[i:, i:]
        nonzero_indices = np.argwhere(submatrix)
        
        if nonzero_indices.size == 0:
            break

        R, C = nonzero_indices[0]

        j = R + i
        k = C + i

        if i != j:
            B[[i, j], :] = B[[j, i], :]
        if i != k:
            B[:, [i, k]] = B[:, [k, i]]

        for h in range(i+1, m):
            if B[h, i] == 1:
                B[h, :] = (B[h, :] + B[i, :]) % 2

        for l in range(i+1, n):
            if B[i, l] == 1:
                B[:, l] = (B[:, l] + B[:, i]) % 2

        i += 1  

    return B

In [21]:
B1=snf_sparse(bound1)

In [22]:
print(B1)

<Compressed Sparse Column sparse matrix of dtype 'int64'
	with 1927 stored elements and shape (1928, 5784)>
  Coords	Values
  (0, 0)	1
  (1, 1)	1
  (2, 2)	1
  (3, 3)	1
  (4, 4)	1
  (5, 5)	1
  (6, 6)	1
  (7, 7)	1
  (8, 8)	1
  (9, 9)	1
  (10, 10)	1
  (11, 11)	1
  (12, 12)	1
  (13, 13)	1
  (14, 14)	1
  (15, 15)	1
  (16, 16)	1
  (17, 17)	1
  (18, 18)	1
  (19, 19)	1
  (20, 20)	1
  (21, 21)	1
  (22, 22)	1
  (23, 23)	1
  (24, 24)	1
  :	:
  (1902, 1902)	1
  (1903, 1903)	1
  (1904, 1904)	1
  (1905, 1905)	1
  (1906, 1906)	1
  (1907, 1907)	1
  (1908, 1908)	1
  (1909, 1909)	1
  (1910, 1910)	1
  (1911, 1911)	1
  (1912, 1912)	1
  (1913, 1913)	1
  (1914, 1914)	1
  (1915, 1915)	1
  (1916, 1916)	1
  (1917, 1917)	1
  (1918, 1918)	1
  (1919, 1919)	1
  (1920, 1920)	1
  (1921, 1921)	1
  (1922, 1922)	1
  (1923, 1923)	1
  (1924, 1924)	1
  (1925, 1925)	1
  (1926, 1926)	1


In [23]:
B2=snf_sparse(bound2)

In [24]:
print(B2)

<Compressed Sparse Column sparse matrix of dtype 'int64'
	with 3855 stored elements and shape (5784, 3856)>
  Coords	Values
  (0, 0)	1
  (1, 1)	1
  (2, 2)	1
  (3, 3)	1
  (4, 4)	1
  (5, 5)	1
  (6, 6)	1
  (7, 7)	1
  (8, 8)	1
  (9, 9)	1
  (10, 10)	1
  (11, 11)	1
  (12, 12)	1
  (13, 13)	1
  (14, 14)	1
  (15, 15)	1
  (16, 16)	1
  (17, 17)	1
  (18, 18)	1
  (19, 19)	1
  (20, 20)	1
  (21, 21)	1
  (22, 22)	1
  (23, 23)	1
  (24, 24)	1
  :	:
  (3830, 3830)	1
  (3831, 3831)	1
  (3832, 3832)	1
  (3833, 3833)	1
  (3834, 3834)	1
  (3835, 3835)	1
  (3836, 3836)	1
  (3837, 3837)	1
  (3838, 3838)	1
  (3839, 3839)	1
  (3840, 3840)	1
  (3841, 3841)	1
  (3842, 3842)	1
  (3843, 3843)	1
  (3844, 3844)	1
  (3845, 3845)	1
  (3846, 3846)	1
  (3847, 3847)	1
  (3848, 3848)	1
  (3849, 3849)	1
  (3850, 3850)	1
  (3851, 3851)	1
  (3852, 3852)	1
  (3853, 3853)	1
  (3854, 3854)	1


In [25]:
z0 = 1928
z1 = 5784 - 1927
z2 = 1
b0 = 1927
b1 = 3855
b2 = 0

In [26]:
betti1 = z0 - b0
betti2 = z1 - b1
betti3 = z2 - b2

In [27]:
print(betti1)
print(betti2)
print(betti3)

1
2
1
